## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Clinical De-identification for French](https://aws.amazon.com/marketplace/pp/prodview-yiju64qx2hokm).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification French

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: [fr.deid_obfuscated](https://nlp.johnsnowlabs.com/2023/06/17/clinical_deidentification_fr.html)
- **Model Description**: This pipeline can be used to deidentify PHI information from medical texts in French. The pipeline can mask and obfuscate the following entities: DATE, AGE, SEX, PROFESSION, ORGANIZATION, PHONE, E-MAIL, ZIP, STREET, CITY, COUNTRY, PATIENT, DOCTOR, HOSPITAL, MEDICALRECORD, SSN, IDNUM, ACCOUNT, PLATE, USERNAME, URL, and IPADDR.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "fr-deid-obfuscated"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**PRENOM : Éric, NOM : Lejeune, NUMÉRO DE SÉCURITÉ SOCIALE : 2730235238020, ADRESSE : 74 Boulevard Riou, VILLE : Sainte Annenec**"

    - **masked**: Default policy that masks entities with their type.

      -> 'PRENOM : `<PATIENT>`, NOM : `<PATIENT>`, NUMÉRO DE SÉCURITÉ SOCIALE : `<SSN>`, ADRESSE : `<STREET>`, VILLE : `<CITY>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'PRENOM : `Mlle Goncalves`, NOM : `Mme Garnier-Roussel`, NUMÉRO DE SÉCURITÉ SOCIALE : `129022554007724`, ADRESSE : `avenue de Mahe`, VILLE : `Meyer-sur-Lemoine`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'PRENOM : `****`, NOM : `****`, NUMÉRO DE SÉCURITÉ SOCIALE : `****`, ADRESSE : `****`, VILLE : `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'PRENOM : [`**`], NOM : [`*****`], NUMÉRO DE SÉCURITÉ SOCIALE : [`***********`], ADRESSE : [`***************`], VILLE : [`************`]'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
'''PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058, ADRESSE : 18 Avenue Matabia... 
PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058, ADRESSE : 18 Avenue Matabiau, VILLE : Grenoble
CODE POSTAL : 38000, DATE DE NAISSANCE : 03/03/1946, Âge : 70 ans , Sexe : H, NUMÉRO DE TÉLÉPHONE : 06-777413307, COURRIEL : jdubois@hotmail.fr, DATE D'ADMISSION : 12/12/2016
MÉDÉCIN : Dr Michel Renaud, RAPPORT CLINIQUE : 70 ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour.
Jean Dubois nous a été adressé car il présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale.
L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV.
L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.
Hémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; les fonctions hépatiques et rénales étaient normales. PSA de 1,16 ng/ml.
ADDRESSÉ À : Dre Marie Breton - Centre Hospitalier de Bellevue Service D'Endocrinologie et de Nutrition - Rue Paulin Bussières, 38000 Grenoble
COURRIEL : mariebreton@chb.fr
NUMÉRO DE TÉLÉPHONE : 06-34567890''',

'''PRENOM : Éric, NOM : Lejeune, NUMÉRO DE SÉCURITÉ SOCIALE : 2730235238020, ADRESSE : 74 Boulevard Rio... 
PRENOM : Éric, NOM : Lejeune, NUMÉRO DE SÉCURITÉ SOCIALE : 2730235238020, ADRESSE : 74 Boulevard Riou, VILLE : Sainte Annenec
CODE POSTAL : 49722, DATE DE NAISSANCE : 10/06/1980, Âge : 38 ans , Sexe : H, NUMÉRO DE TÉLÉPHONE : 06-90123456. COURRIEL : elej@free.fr, DATE D'ADMISSION : 08/11/2018
MÉDÉCIN : Dre Martine Da Costa, RAPPORT CLINIQUE : 
Antécédents : Éric Lejeune de 38 ans, évalué en consultation pour une coxalgie droite d'un an et demi d'évolution, de caractéristiques mécaniques. Éric Lejeune a signalé une douleur à l'exercice et une légère limitation de la mobilité dans certaines postures. Mécanicien de profession, pendant son temps libre, Éric Lejeune faisait du vélo (40 km par jour le week-end), courait 3 fois par semaine pendant 50 minutes et marchait pendant une heure les jours où il ne courait pas. La douleur a progressivement augmenté et elle ne fait actuellement que marcher.
EXAMEN PHYSIQUE : L'examen physique a révélé une douleur à la palpation dans le tiers moyen de la région inguinale droite avec une douleur à la flexion, à l'adduction et à la rotation, surtout à la rotation interne (manœuvre de choc positif). L'équilibre de l'articulation de la hanche est de 110o de flexion bilatérale, 10o de rotation interne droite, 40o de rotation interne gauche et 30o de rotation externe droite, 40o de rotation externe gauche. L'équilibre musculaire sur l'échelle de Daniels est 5/5 global bilatéral. Pas de douleur à la palpation des points sciatiques et les manœuvres de Valleix, Fabere, Lasegue et Bragard sont négatives. Pas non plus de douleur à la palpation de la bandelette iliotibiale et le test d'Ober était négatif. A l'examen de la marche, nous avons observé une absence de claudication et la posture monopodale alternée droite-gauche était stable, avec un Trendelenburg négatif. Le patient avait des douleurs lors de la marche avec une flexion forcée de la hanche et du genou.
Examens complémentaires : Une radiologie ordinaire antéropostérieure (AP) a montré une déformation en forme de bosse dans la région fémorale cervicale droite. L'hémogramme, la biochimie, les hormones thyroïdiennes, les anticorps et les réactifs de phase aiguë requis étaient normaux.
Au vu de ces résultats, une IRM de la hanche a été demandée, qui a montré de légères modifications sous-chondrales, avec un œdème osseux dans la hanche droite, avec une irrégularité corticale et une légère synovite dans l'interligne articulaire, avec une distension de la bourse ilio-psoas. La tête fémorale présente un petit îlot osseux et l'angle alpha est supérieur à 50o. L'arthrographie montre un pincement de l'interligne articulaire antéro-supérieur et postéro-inférieur, un col fémoral ostéophytique naissant avec de petites lésions sous-chondrales antéro-supérieures de la tête fémorale et de petits foyers d'œdème dans l'acétabulum antéro-supérieur. Pas de nécrose avasculaire ni de lésions du labrum. Une lésion kystique juxta-articulaire est également observée en regard du grand trochanter.
Evolution : Avec le diagnostic de syndrome de conflit fémoroacétabulaire de type LEVA (également connu sous le nom de CAM), le patient s'est vu prescrire un traitement par magnétothérapie (15 séances) pour améliorer les symptômes de la douleur et le trophisme osseux.
Éric Lejeune a été adressé à l'unité d'arthroscopie du service de traumatologie pour une évaluation de la chirurgie arthroscopique.
Diagnostic principal : syndrome du conflit fémoroacétabulaire de type LEVA.
TRAITEMENT : Un traitement pharmacologique est prescrit avec une combinaison de glucosamine-chondroïtine sulfate par voie orale 400 mg toutes les 12 heures pendant 3 mois et des anti-inflammatoires non stéroïdiens si nécessaire en cas de douleur. Elle a pour consigne de se reposer dans les sports pour éviter tout choc à l'articulation, avec une activité active contrôlée, en recommandant de ne pas dépasser 90 degrés de flexion de la hanche et d'éviter la rotation. Prenez rendez-vous pour un suivi une fois les sulfates de glucosamine-chondroïtine terminés afin de constater une amélioration clinique. Appelez le numéro des soins et des rendez-vous précédents de l'hôpital : 0470892301.
ADDRESSÉ À : Pr Diane Martinez - Centre Hospitalier Saint Louis - 30 Rue Garcia, 49000 Angers
COURRIEL : dmartinez@chsl.fr
NUMÉRO DE TÉLÉPHONE : 06-44006512'''

]


sample_text = """PRENOM : Aurélie
NOM : Perrier
NUMÉRO DE SÉCURITÉ SOCIALE : 2190150502008
ADRESSE : 8 Rue Faivre
VIL... 
PRENOM : Aurélie
NOM : Perrier
NUMÉRO DE SÉCURITÉ SOCIALE : 2190150502008
ADRESSE : 8 Rue Faivre
VILLE : Bousquet
CODE POSTAL : 11631
DATE DE NAISSANCE : 22/10/1960
Âge : 44 ans 
Sexe : Femme
COURRIEL : aurelieprr@orange.fr
DATE D'ADMISSION : 31/12/2004
MÉDÉCIN : Dr Alain Bouvet
RAPPORT CLINIQUE : 
Motif de l'admission : Aurélie Perrier âgée de 44 ans, adressée à notre unité en raison d'une augmentation de volume dans la région zygomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. Les antécédents médicaux n'étaient pas pertinents. 
Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde.
Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures.
L'étude histopathologique a signalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. Les contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée à 10 mois, avec un remodelage presque complet de la déformation des joues.
ADDRESSÉ À : Dr Joseph Pasquier - Centre Hospitalier de Carcassonne - 1060 Chemin de la Madeleine, 11000 Carcassonne
NUMÉRO DE TÉLÉPHONE : 06-71105388
COURRIEL : pasquier@orange.fr"""

### JSON

#### Example 1: masked (default-policy)

In [9]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json)

,predictions
0,"PRENOM : <PATIENT> NOM : <PATIENT> NUMÉRO DE SÉCURITÉ SOCIALE : <SSN> ADRESSE : <CITY>. .. \nPRENOM : <PATIENT> NOM : <PATIENT> NUMÉRO DE SÉCURITÉ SOCIALE : <SSN> ADRESSE : <STREET> VILLE : <CITY> CODE POSTAL : <ZIP> DATE DE NAISSANCE : <DATE> Âge : <AGE> Sexe : <SEX> COURRIEL : <E-MAIL>\nDATE D'ADMISSION : <DATE> MÉDÉCIN : <DOCTOR> RAPPORT CLINIQUE : \nMotif de l'admission : <PATIENT> âgée de <AGE>ans, adressée à notre unité en raison d'une augmentation de volume dans la région zygomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. <CITY> antécédents médicaux n'étaient pas pertinents. Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde. Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures. L'étude histopathologique a signalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. <CITY> contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée à 10 mois, avec un remodelage presque complet de la déformation des joues. ADDRESSÉ À : <DOCTOR> - <HOSPITAL>, <ZIP><CITY> NUMÉRO DE TÉLÉPHONE : <PHONE> COURRIEL : <E-MAIL>. fr"


#### Example 2: obfuscated

In [11]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json)

,predictions
0,"PRENOM : Mme Lacombe NOM : M. Besson NUMÉRO DE SÉCURITÉ SOCIALE : 1869829291997 ADRESSE : Faivre-sur-Andre. .. \nPRENOM : Mme Lacombe NOM : M. Besson NUMÉRO DE SÉCURITÉ SOCIALE : 1869829291997 ADRESSE : 77, rue Timothée Bourgeois VILLE : Michaud-sur-Carpentier CODE POSTAL : 11831 DATE DE NAISSANCE : 22/10/1960 Âge : 48 ans Sexe : Homme COURRIEL : margotpichon@ifrance.com\nDATE D'ADMISSION : 31/12/2004 MÉDÉCIN : Dr. Nicolas RAPPORT CLINIQUE : \nMotif de l'admission : Mme Thibaut du Hebert âgée de 48ans, adressée à notre unité en raison d'une augmentation de volume dans la région zygomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. Saint Nicolas antécédents médicaux n'étaient pas pertinents. Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde. Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures. L'étude histopathologique a signalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. Saint Nicolas contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée à 10 mois, avec un remodelage presque complet de la déformation des joues. ADDRESSÉ À : Dre. Claude-Margaux Charrier - CENTRE HOSPITALIER ALPES ISERE, 11222Potier NUMÉRO DE TÉLÉPHONE : 84-37781966 COURRIEL : weissolivier@noos.fr. fr"


### JSON Lines

In [13]:
def create_jsonl(records, masking_policy=None):
    json_records = []

    if isinstance(records, str):
        records = [records]

    for text in records:
        record = {"text": text}

        if masking_policy is not None:
            record["masking_policy"] = masking_policy
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record, ensure_ascii=False) for record in json_records)
    return json_lines


#### Example 1: masked (default-policy)

In [14]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="masked")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "PRENOM : <PATIENT> NOM : <PATIENT> NUMÉRO DE SÉCURITÉ SOCIALE : <SSN> ADRESSE : <CITY>. .. \nPRENOM : <PATIENT> NOM : <PATIENT> NUMÉRO DE SÉCURITÉ SOCIALE : <SSN> ADRESSE : <STREET> VILLE : <CITY> CODE POSTAL : <ZIP> DATE DE NAISSANCE : <DATE> Âge : <AGE> Sexe : <SEX> COURRIEL : <E-MAIL>\nDATE D'ADMISSION : <DATE> MÉDÉCIN : <DOCTOR> RAPPORT CLINIQUE : \nMotif de l'admission : <PATIENT> âgée de <AGE>ans, adressée à notre unité en raison d'une augmentation de volume dans la région zygomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. <CITY> antécédents médicaux n'étaient pas pertinents. Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l

#### Example 2: obfuscated

In [16]:
input_jsonl_data = create_jsonl(sample_text, masking_policy="obfuscated")
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": "PRENOM : Mme Lacombe NOM : M. Besson NUMÉRO DE SÉCURITÉ SOCIALE : 1869829291997 ADRESSE : Faivre-sur-Andre. .. \nPRENOM : Mme Lacombe NOM : M. Besson NUMÉRO DE SÉCURITÉ SOCIALE : 1869829291997 ADRESSE : 77, rue Timothée Bourgeois VILLE : Michaud-sur-Carpentier CODE POSTAL : 11831 DATE DE NAISSANCE : 22/10/1960 Âge : 48 ans Sexe : Homme COURRIEL : margotpichon@ifrance.com\nDATE D'ADMISSION : 31/12/2004 MÉDÉCIN : Dr. Nicolas RAPPORT CLINIQUE : \nMotif de l'admission : Mme Thibaut du Hebert âgée de 48ans, adressée à notre unité en raison d'une augmentation de volume dans la région zygomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. Saint Nicolas antécédents médicaux n'étaient pas pertinents. Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [19]:
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [20]:
# Create JSON and JSON Lines data
input_json_data = {
    "input1.json": json.dumps({"text": docs, "masking_policy": "masked"}, ensure_ascii=False),
    "input2.json": json.dumps({"text": docs, "masking_policy": "obfuscated"}, ensure_ascii=False),
    "input3.json": json.dumps({"text": docs, "masking_policy": "masked_fixed_length_chars"}, ensure_ascii=False),
    "input4.json": json.dumps({"text": docs, "masking_policy": "masked_with_chars"}, ensure_ascii=False),
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}

# Upload JSON and JSON Lines data to S3
for file_name, data in input_json_data.items():
    upload_to_s3(data, file_name)

for file_name, data in input_jsonl_data.items():
    upload_to_s3(data, file_name)


### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [23]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    retrieve_json_output_from_s3(validation_file_name)
    print("\n")

Masking policy: masked


{'predictions': ["PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>, ADRESSE : 18 Avenue Matabia. .. \nPRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>, ADRESSE : <STREET>, VILLE : <CITY> CODE POSTAL : <ZIP> DATE DE NAISSANCE : <DATE>, Âge : <AGE> , Sexe : <SEX>, NUMÉRO DE TÉLÉPHONE : <PHONE>, COURRIEL : <E-MAIL>, DATE D'ADMISSION : <DATE> MÉDÉCIN : <DOCTOR>, RAPPORT CLINIQUE : <AGE>ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. <PATIENT> nous a été adressé car <SEX> présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec u



Masking policy: obfuscated


{'predictions': ["PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : 0691071560149, ADRESSE : 18 Avenue Matabia. .. \nPRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : 0691071560149, ADRESSE : boulevard de Vallet, VILLE : Sainte Martin CODE POSTAL : 18000 DATE DE NAISSANCE : 03/03/1946, Âge : 75 ans , Sexe : Femme, NUMÉRO DE TÉLÉPHONE : 62-111057761, COURRIEL : georgeslemonnier@live.com, DATE D'ADMISSION : 12/12/2016 MÉDÉCIN : Dr. Sauvage, RAPPORT CLINIQUE : 75ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. Mme Alexandrie Brun nous a été adressé car Homme présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec 



Masking policy: masked_fixed_length_chars


{'predictions': ["PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : ****, ADRESSE : 18 Avenue Matabia. .. \nPRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : ****, ADRESSE : ****, VILLE : **** CODE POSTAL : **** DATE DE NAISSANCE : ****, Âge : **** , Sexe : ****, NUMÉRO DE TÉLÉPHONE : ****, COURRIEL : ****, DATE D'ADMISSION : **** MÉDÉCIN : ****, RAPPORT CLINIQUE : ****ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. **** nous a été adressé car **** présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux



Masking policy: masked_with_chars


{'predictions': ["PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : [***********], ADRESSE : 18 Avenue Matabia. .. \nPRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : [***********], ADRESSE : [****************], VILLE : [******] CODE POSTAL : [***] DATE DE NAISSANCE : [********], Âge : [****] , Sexe : *, NUMÉRO DE TÉLÉPHONE : [**********], COURRIEL : [****************], DATE D'ADMISSION : [********] MÉDÉCIN : [**************], RAPPORT CLINIQUE : **ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. [*********] nous a été adressé car ** présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [26]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    retrieve_jsonlines_output_from_s3(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "PRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>, ADRESSE : 18 Avenue Matabia. .. \nPRENOM : Jean, NOM : Dubois, NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>, ADRESSE : <STREET>, VILLE : <CITY> CODE POSTAL : <ZIP> DATE DE NAISSANCE : <DATE>, Âge : <AGE> , Sexe : <SEX>, NUMÉRO DE TÉLÉPHONE : <PHONE>, COURRIEL : <E-MAIL>, DATE D'ADMISSION : <DATE> MÉDÉCIN : <DOCTOR>, RAPPORT CLINIQUE : <AGE>ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour. <PATIENT> nous a été adressé car <SEX> présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale. L'examen physique a montré un bo

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

